In [9]:
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import os
from os import listdir
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import random
import math
import csv

imsize=56#define how big the square image should be along a side, e.g. imsize=100 means that images will be converted to 100x100 images, this value should be divisible by 4
num_epochs=200


########################################################################################################################################
#Obtain all images from the data directory, Yale Face Database obtained from http://vision.ucsd.edu/content/yale-face-database
datadirectory=os.getcwd()+'/data/'
allimgnames=os.listdir(datadirectory)

allimgnamesdict=dict()
for i in allimgnames:
    subjnum=int(i[7:9])
    if subjnum in allimgnamesdict.keys():
        allimgnamesdict[subjnum].append(i)
    else:
        allimgnamesdict[subjnum]=list()
        allimgnamesdict[subjnum].append(i)
n_classes=len(allimgnamesdict)
########################################################################################################################################

train_set_names=list()
train_set_labels=list()
test_set_names=list()
test_set_labels=list()

for i in  allimgnamesdict:
    random.shuffle(allimgnamesdict[i])
    hm_test=int(0.2*len(allimgnamesdict[i]))#how many examples to take for the test set, here we use 20% of data for test and 80% for training
    test_set_names=test_set_names+allimgnamesdict[i][0:hm_test]
    test_set_labels=test_set_labels+[int(i)]*hm_test
    train_set_names=train_set_names+allimgnamesdict[i][hm_test:]
    train_set_labels=train_set_labels+[int(i)]*(len(allimgnamesdict[i])-hm_test)

##################################################
#shuffle the training and testing sets 
combinetrain=list(zip(train_set_names,train_set_labels))
combinetest=list(zip(test_set_names,test_set_labels))

random.shuffle(combinetrain)
random.shuffle(combinetest)

train_set_names[:],train_set_labels[:]=zip(*combinetrain)
test_set_names[:],test_set_labels[:]=zip(*combinetest)
##################################################

'''
print allimgtensor
print type(allimgtensor)
print allimglabels
print type(allimaglabelstensor)
'''

testarray=np.zeros((len(test_set_names),imsize*imsize))
testlabels=np.zeros((len(test_set_names),n_classes))
counter=0
for i in test_set_names:
    testlabels[counter,int(test_set_names[counter][7:9])-1]=1#labels as one hot
    im=Image.open(datadirectory+i)
    im=im.resize((imsize,imsize),Image.NEAREST)
    im=np.asarray(im)/255.0
    testarray[counter,:]=im.reshape((imsize*imsize,))
    counter+=1
    '''
    plt.imshow(im)
    plt.show()
    '''

trainarray=np.zeros((len(train_set_names),imsize*imsize))
trainlabels=np.zeros((len(train_set_labels),n_classes))
counter=0
for i in train_set_names:
    trainlabels[counter,int(train_set_names[counter][7:9])-1]=1#labels as one hot
    im=Image.open(datadirectory+i)
    im=im.resize((imsize,imsize),Image.NEAREST)
    im=np.asarray(im)/255.0
    trainarray[counter,:]=im.reshape((imsize*imsize,))
    counter+=1

########################################################################################################################################


In [10]:
grid_search = [(1.0,'max','Adam'),(0.8,'max','Adam'),(0.5,'max','Adam'),(1.0,'avg','Adam'),(0.8,'avg','Adam'),(0.5,'avg','Adam')]
for i in range(len(grid_search)):
    x = tf.placeholder('float', [None, imsize*imsize])
    y = tf.placeholder('float',[None,n_classes])
    keep_rate = grid_search[i][0]
    pool_method = grid_search[i][1]
    optimizer_method = grid_search[i][2]
    keep_prob = tf.placeholder(tf.float32)

    def train_neural_network(x):
        if x == None:
                print('problem')
        def convolutional_neural_network(x):
            weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,1,32])),
                       'W_conv2':tf.Variable(tf.random_normal([5,5,32,64])),
                       'W_fc':tf.Variable(tf.random_normal([int(imsize/4)*int(imsize/4)*64,1024])),#imsize/4 because each max pool has a stride of 2, so it reduces dimensionality by 2 twice or 1/2^2
                       'out':tf.Variable(tf.random_normal([1024, n_classes]))}

            biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
                       'b_conv2':tf.Variable(tf.random_normal([64])),
                       'b_fc':tf.Variable(tf.random_normal([1024])),
                       'out':tf.Variable(tf.random_normal([n_classes]))}
            x = tf.reshape(x, shape=[-1, imsize, imsize, 1])
            conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])
            conv1 = pool2d(conv1,pool_method)

            conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])
            conv2 = pool2d(conv2,pool_method)

            fc = tf.reshape(conv2,[-1, int(imsize/4)*int(imsize/4)*64])#imsize/4 because each max pool has a stride of 2, so it reduces dimensionality by 2 twice or 1/2^2
            fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
            fc = tf.nn.dropout(fc, keep_rate)

            output = tf.matmul(fc, weights['out'])+biases['out']

            return output
    
        def conv2d(x, W):
            return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

        def pool2d(x,how):
            if how == 'max':
            #                        size of window         movement of window
                return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
            #if how == 'avg':
            return tf.nn.avg_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
#------------------------------------------------
        optimizer = None
        prediction = convolutional_neural_network(x)
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
        if optimizer_method == 'Adam':
            optimizer = tf.train.AdamOptimizer().minimize(cost) #learning_rate=0.001
        with tf.Session() as sess:
            #sess.run(tf.initialize_all_variables())
            sess.run(tf.global_variables_initializer())

            acclist=list()
            losslist=list()
            for epoch in range(num_epochs):
                epoch_loss = 0
                _, c = sess.run([optimizer, cost], feed_dict={x: trainarray, y: trainlabels})
                epoch_loss += c
                correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
                accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
                acclist.append(accuracy.eval({x:testarray, y:testlabels}))
                losslist.append(epoch_loss)
                if epoch%40 ==0:
                    print('Epoch '+ str(epoch+1) + '/'+str(num_epochs)+','+' loss:'+str(epoch_loss),'Test Accuracy:',accuracy.eval({x:testarray, y:testlabels}))

            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
            print('Accuracy:',accuracy.eval({x:testarray, y:testlabels}))
            return acclist,losslist
        
    acclist,losslist=train_neural_network(x)
    
    with open('accuracy_dropout_{}_{}pool_{}'.format(keep_rate,pool_method,optimizer_method), 'w') as f:
        for item in acclist:
            f.write("%s\n" % item)

    with open('loss_dropout_{}_{}pool_{}'.format(keep_rate,pool_method,optimizer_method), 'w') as f:
        for item in losslist:
            f.write("%s\n" % item)
print('success')

Epoch 1/200, loss:410572.5 Test Accuracy: 0.16666667
Epoch 41/200, loss:1805.3455810546875 Test Accuracy: 0.8333333
Epoch 81/200, loss:0.0 Test Accuracy: 0.8666667
Epoch 121/200, loss:0.0 Test Accuracy: 0.8666667
Epoch 161/200, loss:0.0 Test Accuracy: 0.8666667
Accuracy: 0.8666667
Epoch 1/200, loss:324389.78125 Test Accuracy: 0.13333334
Epoch 41/200, loss:11893.291015625 Test Accuracy: 0.73333335
Epoch 81/200, loss:856.99462890625 Test Accuracy: 0.76666665
Epoch 121/200, loss:141.6226806640625 Test Accuracy: 0.93333334
Epoch 161/200, loss:96.70977783203125 Test Accuracy: 0.9
Accuracy: 0.8666667
Epoch 1/200, loss:497229.40625 Test Accuracy: 0.033333335
Epoch 41/200, loss:28539.080078125 Test Accuracy: 0.6
Epoch 81/200, loss:999.2918090820312 Test Accuracy: 0.8333333
Epoch 121/200, loss:969.0806274414062 Test Accuracy: 0.8333333
Epoch 161/200, loss:832.826904296875 Test Accuracy: 0.8333333
Accuracy: 0.93333334
Epoch 1/200, loss:477959.8125 Test Accuracy: 0.1
Epoch 41/200, loss:2675.96044